In [1]:
# !git clone git@github.com:cisagov/vulnrichment.git    

For Vulnrichment CVEs, (and in general all CVEs), there may be errors in assigned CWEs per https://github.com/cisagov/vulnrichment/pull/62.

To automate checking of this:
1. Clone Vulnrichment JSON files
2. Extract the CVE Description, and CWE info from the Vulnrichment JSON files to a CSV file
3. 

In [1]:
import os
import json
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


## Read CWE values
These can be in one or both
* ADP container 
* CNA container 

Examples
* 2024/32xxx/CVE-2024-32017.json
    * https://github.com/cisagov/vulnrichment/blob/4380ad1a5f932ba6a29fd01dc825d03a9547196d/2024/32xxx/CVE-2024-32017.json#L19 has CWE-120 in the CNA container.
*  2024/0xxx/CVE-2024-0042.json has 
     * 'unknown' in the CNA container https://github.com/cisagov/vulnrichment/blob/7a8e01764e5ae28d6ef713ecf7c12b9d618c6254/2024/0xxx/CVE-2024-0042.json#L25
     * CWE-843 in the ADP container https://github.com/cisagov/vulnrichment/blob/7a8e01764e5ae28d6ef713ecf7c12b9d618c6254/2024/0xxx/CVE-2024-0042.json#L119

In [ ]:
# Directory containing JSON files
root_directory = '../tmp/vulnrichment'

In [4]:
import os
import json
import pandas as pd

# Function to extract CWE info from a single JSON file
def extract_cwe_info_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        cwe_info_list = []
        filename = os.path.basename(file_path)  # Get filename
        cve_id = os.path.splitext(os.path.basename(file_path))[0]
        
        # Extract from both CNA and ADP containers
        containers = data.get('containers', {})
        if containers:
            for container_name in ['cna', 'adp']:
                container_data = containers.get(container_name, [])
                if container_name == 'cna':
                    cwe_info_list.extend(extract_cwe_info(container_data, filename, prefix="CNA"))
                elif container_name == 'adp':
                    cwe_info_list.extend(extract_cwe_info(container_data, filename, prefix="ADP"))

            # Extract CVE description from CNA container
            cna_section = containers.get('cna', {})
            if cna_section:
                cve_description = extract_cve_description(cna_section)
                for info in cwe_info_list:
                    info['CVE_Description'] = cve_description

        return cwe_info_list

# Unified function to extract CWE info from any container section with a prefix
def extract_cwe_info(container_data, filename, prefix):
    
    cve_id = os.path.splitext(os.path.basename(filename))[0]  # Extract filename without extension as cveId

    cwe_info = []
    
    if isinstance(container_data, dict):
        container_data = [container_data]  # Ensure it is iterable
    
    for entry in container_data:
        problem_types = entry.get('problemTypes', [])
        for problem in problem_types:
            descriptions = problem.get('descriptions', [])
            for description in descriptions:
                cwe_id = description.get('cweId')
                cwe_description = description.get('description')
                if cwe_id and cwe_description:
                    cwe_info.append({
                        'cve_id': cve_id,
                        f'{prefix}_CWE_ID': cwe_id,
                        f'{prefix}_CWE_Description': cwe_description
                    })
    return cwe_info

# Function to extract CVE description from CNA container
def extract_cve_description(cna_data):
    descriptions = cna_data.get('descriptions', [])
    for description in descriptions:
        if description.get('lang') == 'en':
            return description.get('value', '')
    return ''

# Function to extract CWE info from all JSON files in a directory
def extract_cwe_info_from_directory(root_dir):
    all_cwe_info = []
    for root, _, files in os.walk(root_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                file_path = os.path.join(root, file_name)
                cwe_info = extract_cwe_info_from_json(file_path)
                if cwe_info:
                    all_cwe_info.extend(cwe_info)
    return all_cwe_info


# Extract CWE info from directory
cwe_info_list = extract_cwe_info_from_directory(root_directory)

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(cwe_info_list)

# Save the DataFrame to a CSV file
df.to_csv('../data_out/vulnrichment/cwe_info.csv', index=False)


In [5]:
df['CVE_Description'] = df['CVE_Description'].str.replace('\n', ' ')

In [6]:
df

,cve_id,ADP_CWE_ID,ADP_CWE_Description,CVE_Description,CNA_CWE_ID,CNA_CWE_Description
0,CVE-2013-3245,CWE-125,CWE-125 Out-of-bounds Read,plugins/demux/libmkv_plugin.dll in VideoLAN VL...,NaN,NaN
1,CVE-2013-3245,CWE-122,CWE-122 Heap-based Buffer Overflow,plugins/demux/libmkv_plugin.dll in VideoLAN VL...,NaN,NaN
2,CVE-2014-0808,CWE-566,CWE-566 Authorization Bypass Through User-Cont...,Authorization bypass through user-controlled k...,NaN,NaN
3,CVE-2014-5470,CWE-77,CWE-77 Improper Neutralization of Special Elem...,Actual Analyzer through 2014-08-29 allows code...,NaN,NaN
4,CVE-2022-2586,NaN,NaN,It was discovered that a nft object or express...,CWE-416,CWE-416
...,...,...,...,...,...,...
7519,CVE-2018-4233,CWE-119,CWE-119 Improper Restriction of Operations wit...,An issue was discovered in certain Apple produ...,NaN,NaN
7520,CVE-2018-19093,CWE-122,CWE-122 Heap-based Buffer Overflow,An issue has been found in libIEC61850 v1.3. I...,NaN,NaN
7521,CVE-2011-0611,CWE-843,CWE-843 Access of Resource Using Incompatible ...,Adobe Flash Player before 10.2.154.27 on Windo...,NaN,NaN
7522,CVE-2011-0737,CWE-200,CWE-200 Exposure of Sensitive Information to a...,Adobe ColdFusion 9.0.1 CHF1 and earlier allows...,NaN,NaN


In [7]:
df.ADP_CWE_ID.isna().value_counts()

True     6113
False    1411
Name: ADP_CWE_ID, dtype: int64

In [8]:
df.CNA_CWE_ID.isna().value_counts()

False    6113
True     1411
Name: CNA_CWE_ID, dtype: int64

# Now Ask an LLM to Validate 

1. Assign a CWE based on Description
2. Provide say if it agrees with the assigned CWE

In [2]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
openai.Model.list()

ModuleNotFoundError: No module named 'openai'

In [9]:
cwe_info_list = cwe_info_list[:10]


In [29]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [30]:
# Function to get ChatGPT-4's CWE assignment and rationale
def get_chatgpt4_cwe_assignment(cve_description):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a cybersecurity expert specializing in identifying Common Weakness Enumeration (CWE) IDs from CVE descriptions."
            },
            {
                "role": "user",
                "content": f"Based on the following CVE description, assign the most appropriate CWE ID, provide the rationale, and give a confidence score (0-100):\n\n{cve_description}\n\nResponse format:\nCWE ID: <CWE ID>\nRationale: <rationale>\nConfidence score: <score>"
            }
        ],
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

# Function to get ChatGPT-4's opinion on an existing CWE assignment
def get_chatgpt4_cwe_opinion(cve_description, cwe_id, cwe_description):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a cybersecurity expert specializing in identifying Common Weakness Enumeration (CWE) IDs from CVE descriptions."
            },
            {
                "role": "user",
                "content": f"Given the following CVE description, do you agree with the assigned CWE ID '{cwe_id}' and the following description '{cwe_description}'? Provide your rationale and give a confidence score (0-100):\n\n{cve_description}\n\nResponse format:\nAgree: <yes/no>\nRationale: <rationale>\nConfidence score: <score>"
            }
        ],
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

In [31]:
# Function to get ChatGPT-4's CWE assignment and rationale
def get_chatgpt4_cwe_assignment(cve_description):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a cybersecurity expert specializing in identifying Common Weakness Enumeration (CWE) IDs from CVE descriptions."
            },
            {
                "role": "user",
                "content": f"Based on the following CVE description, assign the most appropriate CWE ID, provide the rationale, and give a confidence score (0-100):\n\n{cve_description}\n\nResponse format:\nCWE ID: <CWE ID>\nRationale: <rationale>\nConfidence score: <score>"
            }
        ],
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

# Function to get ChatGPT-4's opinion on an existing CWE assignment
def get_chatgpt4_cwe_opinion(cve_description, cwe_id, cwe_description):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a cybersecurity expert specializing in identifying Common Weakness Enumeration (CWE) IDs from CVE descriptions."
            },
            {
                "role": "user",
                "content": f"Given the following CVE description, do you agree with the assigned CWE ID '{cwe_id}' and the following description '{cwe_description}'? Provide your rationale and give a confidence score (0-100):\n\n{cve_description}\n\nResponse format:\nAgree: <yes/no>\nRationale: <rationale>\nConfidence score: <score>"
            }
        ],
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

In [12]:
cwe_info_list

[{'cve_id': 'CVE-2013-3245',
  'ADP_CWE_ID': 'CWE-125',
  'ADP_CWE_Description': 'CWE-125 Out-of-bounds Read',
  'CVE_Description': 'plugins/demux/libmkv_plugin.dll in VideoLAN VLC Media Player 2.0.7, and possibly other versions, allows remote attackers to cause a denial of service (crash) and possibly execute arbitrary code via a crafted MKV file, possibly involving an integer overflow and out-of-bounds read or heap-based buffer overflow, or an uncaught exception.  NOTE: the vendor disputes the severity and claimed vulnerability type of this issue, stating "This PoC crashes VLC, indeed, but does nothing more... this is not an integer overflow error, but an uncaught exception and I doubt that it is exploitable. This uncaught exception makes VLC abort, not execute random code, on my Linux 64bits machine." A PoC posted by the original researcher shows signs of an attacker-controlled out-of-bounds read, but the affected instruction does not involve a register that directly influences cont

In [34]:
cve_description = "It was discovered that a nft object or expression could reference a nft set on a different nft table, leading to a use-after-free once that table was deleted."
chatgpt4_opinion = get_chatgpt4_cwe_assignment(cve_description)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [32]:

# Adding ChatGPT-4's CWE assignment and opinions
for entry in cwe_info_list:
    cve_description = entry.get('cve_description', '')

    # Get ChatGPT-4's CWE assignment and rationale
    chatgpt4_assignment = get_chatgpt4_cwe_assignment(cve_description)
    entry['ChatGPT4_CWE_Assignment'] = chatgpt4_assignment

    # Get ChatGPT-4's opinion on existing CNA CWE ID
    if 'CNA_CWE_ID' in entry:
        cna_cwe_id = entry['CNA CWE ID']
        cna_cwe_description = entry['CNA_CWE_Description']
        chatgpt4_opinion = get_chatgpt4_cwe_opinion(cve_description, cna_cwe_id, cna_cwe_description)
        entry['ChatGPT4 Opinion on CNA CWE'] = chatgpt4_opinion

    # Get ChatGPT-4's opinion on existing ADP CWE ID
    if 'ADP_CWE_ID' in entry:
        adp_cwe_id = entry['ADP_CWE_ID']
        adp_cwe_description = entry['ADP_CWE_Description']
        chatgpt4_opinion = get_chatgpt4_cwe_opinion(cve_description, adp_cwe_id, adp_cwe_description)
        entry['ChatGPT4 Opinion on ADP CWE'] = chatgpt4_opinion

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(cwe_info_list)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
